# 2.01 - Airline Routes Dataset
## **airline_routes.tsv**
    As of June 2014, the OpenFlights/Airline Route Mapper Route Database contains 67663 routes between 3321 airports on 548 airlines spanning the globe, as shown in the map above. 
- Source: [link](https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat)
- MimeType: /text/csv
- Columns:
    - `Airline`	                2-letter (IATA) or 3-letter (ICAO) code of the airline.
    - `Airline_ID`	            Unique OpenFlights identifier for airline (see Airline).
    - `Source_airport`	        3-letter (IATA) or 4-letter (ICAO) code of the source airport.
    - `Source_airport_ID`	    OpenFlights identifier for source airport    
    - `Destination_airport`	    3-letter (IATA) or 4-letter (ICAO) code of the destination airport.
    - `Destination_airport_ID`	OpenFlights identifier for destination
    - `Codeshare`	            "Y" if this flight is a codeshare (not operated by Airline, but another carrier), empty otherwise.
    - `Stops`                   Number of stops on this flight ("0" for direct)
    - `Equipment`	            3-letter codes for plane type(s) generally used on this flight, separated by spaces




## **planes.tsv**
    The OpenFlights plane database contains a curated selection of 173 passenger aircraft with IATA and/or ICAO codes, covering the vast majority of flights operated today and commonly used in flight schedules and reservation systems. Each entry contains the following information:
- Source: [link](https://raw.githubusercontent.com/jpatokal/openflights/master/data/planes.dat)
- MimeType: /text/csv
- Columns:
    - `Name`	            Full name of the aircraft.
    - `IATA_Code`	        Unique three-letter IATA identifier for the aircraft.
    - `ICAO_Code`	        Unique four-letter ICAO identifier for the aircraft.



## **airpots.tsv**
- Source: [link](https://ourairports.com/data/)
- MimeType: /text/csv
- Columns:
    - `ID`	                description
    - `Ident`	            description
    - `Type`	            description
    - `Name`                description
    - `Latitude_Deg`        description
    - `Longitude_Deg`	    description
    - `Elevation_Ft`	    description
    - `Continent`	        description
    - `ISO_Country`         description
    - `ISO_Region`          description
    - `Municipality`	    description
    - `Scheduled_Service`	description
    - `ICAO_Code`	        description
    - `IATA_Code`           description
    - `GPS_Code`            description
    - `Local_Code`	        description
    - `Home_Link`	        description
    - `Wikepedia_Link`      description
    - `Keywords`            description


In [ ]:
import pandas as pd
import numpy as np
import requests
from io import StringIO

### Airline_routes.tsv

In [ ]:
## URL ## 
routes_url = "https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat"

## Download ##
response = requests.get(routes_url)

columns = [
    "Airline",
    "Airline_ID",
    "Source_Airport",
    "Source_Airport_ID",
    "Destination_Airport",
    "Destination_Airport_ID",
    "Codeshare",
    "Stops",
    "Equipment"
]

## Save as TSV ##
df = pd.read_csv(StringIO(response.text), header=None, names = columns, index_col= False)

df.to_csv('../data/joined_datasets/airline_routes.tsv', sep = '\t', index = False)


### airline_planes.tsv

In [ ]:
# URL of the data
planes_url = "https://raw.githubusercontent.com/jpatokal/openflights/master/data/planes.dat"

# Download the file
response = requests.get(planes_url)

columns = [
    "Name",
    "IATA_Code",
    "ICAO_Code",
]

df = pd.read_csv(StringIO(response.text), header=None, names = columns, index_col= False)
df.to_csv('../data/joined_datasets/airline_planes.tsv', sep = '\t', index = False)


### airlines_airports.tsv

In [ ]:
# Used csv2tab to convert to tsv
df = pd.read_csv('../data/joined_datasets/airline_airports.tsv', sep = "\t")
df.info()

### Combining 3 datasets

In [ ]:
df_airports = pd.read_csv('../data/joined_datasets/airline_airports.tsv', sep = "\t")
df_planes = pd.read_csv('../data/joined_datasets/airline_planes.tsv', sep = "\t")
df_routes = (pd.read_csv('../data/joined_datasets/airline_routes.tsv', sep = "\t")
                .astype({
                    "Airline": str,
                    "Airline_ID": str,
                    "Source_Airport": str,
                    "Source_Airport_ID": str,
                    "Destination_Airport": str,
                    "Destination_Airport_ID": str,
                    "Codeshare": str,
                    "Stops": str, 
                    "Equipment": str   
                })
)
df_routes.head()

In [ ]:
df_routes.columns

In [ ]:
american_airports = (df_airports
                    .loc[df_airports["Iso_Country"] == "US"]
                    .drop(["Continent", "Wikipedia_Link", "Keywords", "Home_Link"], axis = 1)
                    .fillna({
                        "Iata_Code": 0,
                        "Icao_Code": 0,
                        "GPS_Code": 0,
                        "Local_Code": 0
                    })
                    .astype({
                        "Name": str,
                        "Iata_Code": str,
                        "Icao_Code": str,
                        "Gps_Code": str,
                        "Local_Code": str,
                        "Latitude_Deg": float,
                        "Longitude_Deg": float,
                        "Elevation_Ft": float,
                        
                    })
)


american_Iata_Codes = american_airports.loc[american_airports['Iata_Code'] != '0', 'Iata_Code'].to_list()
american_Iata_Codes[:10]

In [ ]:


df_out = df_routes.copy()

## Add Source Longitude, Latitude, Airport Type ##
df_out = df_out.merge(df_airports[["Latitude_Deg","Longitude_Deg","Type", "Iso_Country","Iata_Code"]],
                      left_on = "Source_Airport", 
                      right_on = "Iata_Code",
                      how = 'left',
                      ).rename({"Latitude_Deg": "Latidude_Source", "Longitude_Deg" : "Longitude_Source", "Type" : "Type_Source", "Iso_Country": "Country_Source"}, axis = 1)

## Add Destination Longitude, Latitude, Airport Type ##
df_out = df_out.merge(df_airports[["Latitude_Deg","Longitude_Deg","Type", "Iso_Country","Iata_Code"]],
                      left_on = "Destination_Airport", 
                      right_on = "Iata_Code",
                      how = 'left',
                      ).rename({"Latitude_Deg": "Latidude_Dest", "Longitude_Deg" : "Longitude_Dest", "Type" : "Type_Dest", "Iso_Country": "Country_Dest"}, axis = 1)
df_out.drop(["Iata_Code_x", "Iata_Code_y"], axis = 1, inplace = True)
df_out